**Install Packages**

In [1]:
!pip install torch trochvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement trochvision (from versions: none)
ERROR: No matching distribution found for trochvision


In [2]:
!pip install laserembeddings

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.8/859.8 KB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 KB 5.5 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.35-py3-none-any.whl size=883989 sha256=6e7c2a8702e4792ef60df4ec4ad6ec7b512719bd014a2bf80033cf1517736b27
  Stored in directory: /root/.cache/pip/wheels/c4/df/30/3d6c623db99d503dcdbae1f686953b7c1a0754d8a658dc0845
Successfully built sacremoses


In [3]:
import pandas as pd
import random as rd
import json
import numpy as np
import re
from sklearn.neighbors import NearestNeighbors


In [4]:
!python -m laserembeddings download-models


✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


# Monolingual Embedding

> Indented block



In [5]:
from laserembeddings import Laser

laser = Laser()

# if all sentences are in the same language:

embeddings = laser.embed_sentences(
    ['به ملازمان سلطان که رساند این دعا را که به شکر پادشاهی ز نظر مران گدا را ز رقیب دیوسیرت به خدای خود پناهم مگر آن شهاب ثاقب مددی دهد خدا را',
     'هزار و سیصدتا سلام'],
    lang='fa')  # lang is only used for tokenization

In [6]:
print(embeddings.shape)
print(embeddings)
%cd "/content/"

(2, 1024)
[[ 0.01072003  0.00345858  0.00174004 ...  0.02418301  0.01499301
   0.01545107]
 [ 0.0038445   0.00011095  0.00274064 ...  0.0077541   0.00159712
  -0.00810177]]
/content


# Multilingual Embedding

In [7]:
#Example
embeddings = laser.embed_sentences(
    ['I love pasta.',
     "J'adore les pâtes.",
     'Ich liebe Pasta.'],
    lang=['en', 'fr', 'de'])

**Input data set**

read input data set and creat the input table

In [8]:
data_set_root = "/content/drive/MyDrive/Text_enrich/datasets/"
input_data = pd.read_csv(data_set_root + "input_text/input_text.csv")

In [9]:
columns = ["input_text" , "vector"]
input_table = pd.DataFrame([], columns=columns)
for sample in rd.sample(list(pd.unique(input_data["input_text"])),100):
  input_table = input_table.append({'input_text':sample,'vector':laser.embed_sentences(sample, 'fa')}, ignore_index = True)

In [10]:
input_table

,input_text,vector
0,باید قبل از سخن گفتن فکر کرد.,"[[0.0033361597, -0.00017175634, 0.00010727477,..."
1,با دوستان باید نیکو رفتار کرد,"[[0.0009808637, 0.0017847791, 0.01899858, 0.02..."
2,آخرت را باید بر دنیا ترجیح داد,"[[-0.00022176799, -1.7649661e-07, -2.1464766e-..."
3,باید به قلب و احساسات توجه کرد,"[[0.0009840607, -0.00085574336, 0.001155418, 0..."
4,باید از ستیزه جویی پرهیز کنیم,"[[0.0014077091, -6.465741e-05, 0.00015617054, ..."
...,...,...
95,نباید بی حال و بی حوصله بود,"[[-0.00021816047, -0.00041640573, 0.0006439675..."
96,باید دانشمندان را شناخت و از آن‌ها قدردانی کرد,"[[0.006814094, -0.00036874664, 0.0015877598, 0..."
97,نباید کسانی که مجبوریم به خاطر آن‌ها به تکلف ب...,"[[0.0046849884, -4.4481662e-06, -0.00022142888..."
98,ما باید خودمان را گرامی بداریم.,"[[0.0037770465, -8.119756e-06, 0.057631925, 0...."


# Output data set embedding




English qoutes

creating lightweight data set

In [80]:
columns = ["output_text" , "vector", "output_data_source","index"]
English_qoute_table = pd.DataFrame([], columns=columns)
f = open(data_set_root +'quotes/quotes.jsonl')
new_file = open(data_set_root + "quotes/light_quotes.jsonl", "w")

quotes = [json.loads(jline) for jline in f.read().splitlines()]
n = 0
for i in range(len(quotes)):
  if 20 < len(quotes[i]["quote"]) < 210:
    new_file.write(str(quotes[i])+ '\n')
    n+=1
new_file.close()
f.close()

In [87]:
columns = ["output_text" , "vector", "output_data_source","index"]
English_qoute_table = pd.DataFrame([], columns=columns)
f = open(data_set_root +'quotes/quotes.jsonl')
quotes = [json.loads(jline) for jline in f.read().splitlines()]
n = 0
for i in range(len(quotes)):
  if len(quotes[i]["quote"]) < 210:
    n+=1
    English_qoute_table = English_qoute_table.append({'output_text':quotes[i]["quote"][1:-2],'vector':laser.embed_sentences(quotes[i]["quote"][1:-2], 'en'), 'output_data_source':'نقل قول از نویسندگان انگلیسی', 'index':n}, ignore_index = True)

f.close()

In [88]:
pd.DataFrame(English_qoute_table).to_csv(data_set_root+ "LASER_output/light_quotes.csv")

In [71]:
for k in range(len(input_table)):
  max = 0.0
  index = 0

  for i in range(len(English_qoute_table)):
    image = np.array(input_table["vector"].iloc[k])@np.array(English_qoute_table["vector"].iloc[i]).T
    if(image > max):
      max = image
      index = i

  print(input_table["input_text"].iloc[k] + " => " + English_qoute_table["output_text"].iloc[index] )  
  print("_"*100) 
     

باید قبل از سخن گفتن فکر کرد. => Clary, Despite everything, I can't bear the thought of this ring being lost forever, any more then I can bear the thought of leaving you forever. And though I have no choice about the one, at least I can choose about the other. I'm leaving you our family ring because you have as much right to it as I do.I'm writing this watching the sun come up. You're asleep, dreams moving behind your restless eyelids. I wish I knew what you were thinking. I wish I could slip into your head and see the world the way you do. I wish I could see myself the way you do. But maybe I dont want to see that. Maybe it would make me feel even more than I already do that I'm perpetuating some kind of Great Lie on you, and I couldn't stand that. I belong to you. You could do anything you wanted with me and I would let you. You could ask anything of me and I'd break myself trying to make you happy. My heart tells me this is the best and greatest feeling I have ever had. But my mind 

In [37]:
columns = ["output_text" , "vector", "output_data_source","index"]
Nahj_table = pd.DataFrame([], columns=columns)
file1 = open(data_set_root + 'nahj/Nahj Al-Balaqa.txt', 'r')
Lines = file1.readlines()
nahj = []
count = 1
i = 0
new_file = open(data_set_root + "light_nahj.txt", "w")

for line in Lines:
    splited = line.split("\t")
    i +=1
    if len(splited[1]) < 400:
      new_file. write(str(splited[0]) + "\t" + splited[1])
      # Nahj_table = Nahj_table.append({'output_text':splited[1],'vector':laser.embed_sentences(splited[1], 'ar'), 'output_data_source':'حکمت نهج البلاغه','index':i}, ignore_index = True)
file1.close()
new_file.close()


Nahj Al-Balaqa

In [38]:
columns = ["output_text" , "vector", "output_data_source","index"]
Nahj_table = pd.DataFrame([], columns=columns)
file1 = open(data_set_root + 'nahj/light_nahj.txt', 'r')
Lines = file1.readlines()
nahj = []
count = 1
i = 0
for line in Lines:
    splited = line.split("\t")
    i +=1
    Nahj_table = Nahj_table.append({'output_text':splited[1],'vector':laser.embed_sentences(splited[1], 'ar'), 'output_data_source':'حکمت نهج البلاغه','index':i}, ignore_index = True)
file1.close()


In [15]:
pd.DataFrame(Nahj_table).to_csv(data_set_root+ "LASER_output/nahj.csv")

In [39]:
for k in range(len(input_table)):
  max = 0.0
  index = 0

  for i in range(len(Nahj_table)):
    image = np.array(input_table["vector"].iloc[k])@np.array(Nahj_table["vector"].iloc[i]).T
    if(image > max):
      max = image
      index = i

  print(input_table["input_text"].iloc[k] + " => " + Nahj_table["output_text"].iloc[index] )  
  print("_"*100) 
     

باید قبل از سخن گفتن فکر کرد. => وَ قَالَ ع لَا تَقُل مَا لَا تَعلَمُ بَل لَا تَقُل کُلّ مَا تَعلَمُ فَإِنّ اللّهَ فَرَضَ عَلَی جَوَارِحِکَ کُلّهَا فَرَائِضَ یَحتَجّ بِهَا عَلَیکَ یَومَ القِیَامَةِ

____________________________________________________________________________________________________
با دوستان باید نیکو رفتار کرد => وَ قَالَ ع مَا کَانَ اللّهُ لِیَفتَحَ عَلَی عَبدٍ بَابَ الشّکرِ وَ یُغلِقَ عَنهُ بَابَ الزّیَادَةِ وَ لَا لِیَفتَحَ عَلَی عَبدٍ بَابَ الدّعَاءِ وَ یُغلِقَ عَنهُ بَابَ الإِجَابَةِ وَ لَا لِیَفتَحَ لِعَبدٍ بَابَ التّوبَةِ وَ یُغلِقَ عَنهُ بَابَ المَغفِرَةِ

____________________________________________________________________________________________________
آخرت را باید بر دنیا ترجیح داد => وَ قَالَ ع لِبَعضِ أَصحَابِهِ فِی عِلّةٍ اعتَلّهَا جَعَلَ اللّهُ مَا کَانَ مِن شَکوَاکَ حَطّاً لِسَیّئَاتِکَ فَإِنّ المَرَضَ لَا أَجرَ فِیهِ وَ لَکِنّهُ یَحُطّ السّیّئَاتِ وَ یَحُتّهَا حَتّ الأَورَاقِ وَ إِنّمَا الأَجرُ فِی القَولِ بِاللّسَانِ وَ العَمَلِ باِلأیَدیِ وَ الأَقد

Yek-Beyti

In [17]:
columns = ["output_text" , "vector", "output_data_source","index"]
tak_beit_table = pd.DataFrame([], columns=columns)
file1 = open(data_set_root + 'poems/yek-beiti.txt', 'r')
Lines = file1.readlines()
i = 1
for line in Lines:
    tak_beit_table = tak_beit_table.append({'output_text':line,'vector':laser.embed_sentences(line, 'fa'),'output_data_source':" شعرهای تک بیتی",'index':i}, ignore_index = True)
    i+=1
file1.close()
     


In [18]:
pd.DataFrame(tak_beit_table).to_csv(data_set_root+ "LASER_output/poems.csv")


*Evaluate* 

In [19]:


for k in range(len(input_table)):
  max = 0.0
  index = 0

  for i in range(len(tak_beit_table)):
    image = np.array(input_table["vector"].iloc[k])@np.array(tak_beit_table["vector"].iloc[i]).T
    if(image > max):
      max = image
      index = i

  print(input_table["input_text"].iloc[k] + " => " + tak_beit_table["output_text"].iloc[index] )  
  print("_"*100) 
     


باید قبل از سخن گفتن فکر کرد. => بلا ندیده دعا را شروع باید کرد/ علاج واقعه پیش از وقوع باید کرد

____________________________________________________________________________________________________
با دوستان باید نیکو رفتار کرد => از دشمنان برند شکایت به دوستان/ چون دوست دشمن است شکایت کجا بریم

____________________________________________________________________________________________________
آخرت را باید بر دنیا ترجیح داد => استاد کائنات که این جهان ساخت/ مقصود عشق بود این جهان را بهانه ساخت

____________________________________________________________________________________________________
باید به قلب و احساسات توجه کرد => تا توانی به جهان خدمت محتاجان کن/ به دمی یا درمی یا قدمی یا قلمی

____________________________________________________________________________________________________
باید از ستیزه جویی پرهیز کنیم => قانع به یک استخوان چو کرکس بودن/ به که طفیل خوان ناکس بودن

____________________________________________________________________________________________________
با

غزلیات حافظ







In [20]:
columns = ["output_text" , "vector", " ","index"]
hafez_table = pd.DataFrame([], columns=columns)
file1 = open( data_set_root +'hafez/hafez-qazals.txt', 'r')
Lines = file1.readlines()
flag = False
qazals = []
i = 0
for line in Lines:
    qazal = r".*غزل.*"
    qazal_reg = f"{qazal}"    
    if(re.match(qazal_reg, line) and len(line)<20):
      print(line)
      input =  '\r\n'.join(qazals)
      i += 1
      hafez_table = hafez_table.append({'output_text':input,'vector':laser.embed_sentences(input, 'fa'), 'output_data_source':'غزلیات حافظ', 'index':i}, ignore_index = True)
      qazals = []
    else:
      if(len(line)>5): 
        qazals.append(line.strip())

    
file1.close()

     


﻿غزل    ۱

غزل    ۲

غزل    ۳

غزل   ۴

غزل   ۵

غزل    ۶

غزل    ۷

غزل    ۸

غزل    ۹

غزل    ۱۰

غزل    ۱۰

غزل    ۱۲

غزل    ۱۳

غزل    ۱۴

غزل   ۱۵

غزل    ۱۶

غزل    ۱۷

غزل   ۱۸

غزل    ۱۹

غزل    ۲۰

غزل    ۲۱

غزل    ۲۲

غزل    ۲۳

غزل    ۲۴

غزل    ۲۵

غزل    ۲۶

غزل    ۲۷

غزل    ۲۸

غزل    ۲۹

غزل    ۳۰

غزل    ۳۱

غزل    ۳۲

غزل    ۳۳

غزل   ۳۴

غزل    ۳۵

غزل    ۳۶

غزل    ۳۷

غزل    ۳۸

غزل    ۳۹

غزل    ۴۰

غزل    ۴۱

غزل    ۴۲

غزل    ۴۳

غزل    ۴۴

غزل    ۴۵

غزل    ۴۶

غزل    ۴۷

غزل    ۴۸

غزل    ۴۹

غزل    ۵۰

غزل    ۵۱

غزل    ۵۲

غزل    ۵۳

غزل    ۵۴

غزل    ۵۵

غزل    ۵۶

غزل    ۵۷

غزل    ۵۸

غزل    ۵۹

غزل    ۶۰

غزل    ۶۱

غزل    ۶۲

غزل    ۶۳

غزل    ۶۴

غزل    ۶۵

غزل    ۶۶

غزل    ۶۷

غزل    ۶۸

غزل    ۶۹

غزل    ۷۰

غزل    ۷۱

غزل    ۷۲

غزل    ۷۳

غزل    ۷۴

غزل    ۷۵

غزل    ۷۶

غزل    ۷۷

غزل    ۷۸

غزل    ۷۹

غزل    ۸۰

غزل    ۸۱

غزل    ۸۲

غزل    ۸۳

غزل    ۸۴

غزل    ۸۵

غزل    ۸۶

غزل    ۸۷

غزل    ۸۸

غزل    ۸۹

غزل    ۹۰

غزل    ۹۱

غزل    ۹۲

غ

In [21]:
for k in range(len(input_table)):
  max = 0.0
  index = 0

  for i in range(len(hafez_table)-1):
    image = np.array(input_table["vector"].iloc[k])@np.array(hafez_table["vector"].iloc[i+1]).T
    if(image > max):
      max = image
      index = i+1

  print(input_table["input_text"].iloc[k] + "  ======>>>> " + " غزل شماره " + str(index))  
  print(hafez_table["output_text"].iloc[index] )
  print("_"*100) 

باید قبل از سخن گفتن فکر کرد.  ======>>>>  غزل شماره 285
در عهد پادشاه خطابخش جرم پوش
حافظ قرابه کش شد و مفتی پياله نوش
صوفی ز کنج صومعه با پای خم نشست
تا ديد محتسب که سبو می‌کشد به دوش
احوال شيخ و قاضی و شرب اليهودشان
کردم سال صبحدم از پير می فروش
گفتا نه گفتنيست سخن گر چه محرمی
درکش زبان و پرده نگه دار و می بنوش
ساقی بهار می‌رسد و وجه می‌نماند
فکری بکن که خون دل آمد ز غم به جوش
عشق است و مفلسی و جوانی و نوبهار
عذرم پذير و جرم به ذيل کرم بپوش
تا چند همچو شمع زبان آوری کنی
پروانه مراد رسيد ای محب خموش
ای پادشاه صورت و معنی که مثل تو
ناديده هيچ ديده و نشنيده هيچ گوش
چندان بمان که خرقه ازرق کند قبول
بخت جوانت از فلک پير ژنده پوش
____________________________________________________________________________________________________
با دوستان باید نیکو رفتار کرد  ======>>>>  غزل شماره 244
معاشران گره از زلف يار باز کنيد
شبی خوش است بدين قصه‌اش دراز کنيد
حضور خلوت انس است و دوستان جمعند
و ان يکاد بخوانيد و در فراز کنيد
رباب و چنگ به بانگ بلند می‌گويند
که گوش هوش به پيغام اهل راز کنيد
به جان دو

In [22]:
pd.DataFrame(hafez_table).to_csv(data_set_root+ "LASER_output/hafez.csv")

غزلیات شمس

In [23]:
moulavi_qazals_path = data_set_root + "moulavi/ultra_light_divane-e-shams.txt"
moulavi_qazals_pattern = re.compile(r'\n?\s*\d+\s*?\n')

def read_divan_shams():
    with open(moulavi_qazals_path, encoding='utf-8') as f:
        text = f.read()
        
    qazals_arr = moulavi_qazals_pattern.split(text)
    return qazals_arr[1:]


Creat Ultra light wight and ligthweight dataset

In [55]:
columns = ["output_text" , "vector", "output_data_source","index"]
molavi_table = pd.DataFrame([], columns=columns)
molavi_data_len = len(read_divan_shams())
input_set = read_divan_shams()
new_file = open(data_set_root + "moulavi/light_divane-e-shams.txt", "w")
for i in range(0, molavi_data_len):
  if len(input_set[i]) < 600:
    input = input_set[i]
    new_file. write(str(i) + '\n'+ input + '\n\n')
new_file.close()
    #molavi_table = molavi_table.append({'output_text':input,'vector':laser.embed_sentences(input, 'fa'), 'output_data_source':'غزلیات مولوی', 'index': i+1}, ignore_index = True)

In [51]:
columns = ["output_text" , "vector", "output_data_source","index"]
molavi_table = pd.DataFrame([], columns=columns)
molavi_data_len = len(read_divan_shams())
input_set = read_divan_shams()

for i in range(0, molavi_data_len):
    input = input_set[i]
    molavi_table = molavi_table.append({'output_text':input,'vector':laser.embed_sentences(input, 'fa'), 'output_data_source':'غزلیات مولوی', 'index': i+1}, ignore_index = True)

In [ ]:
print(molavi_table)

In [54]:
# note that these are only the poems that their length is up tp 800 character
pd.DataFrame(molavi_table).to_csv(data_set_root+ "LASER_output/moulavi_800.csv")

In [ ]:
for k in range(len(input_table)):
  max = 0.0
  index = 0

  for i in range(len(molavi_table)-1):
    image = np.array(input_table["vector"].iloc[k])@np.array(molavi_table["vector"].iloc[i+1]).T
    if(image > max):
      max = image
      index = i+1

  print(input_table["input_text"].iloc[k] + "  ======>>>> ")  
  print(molavi_table["output_text"].iloc[index] )
  print("_"*100) 

 تجمیع دادگان خروجی

**Selecting Top N Output for each input**

In [ ]:


def create_pair_knn(k,model_name,output_data_source,input_table,output_table):
  columns = ["model","input_text","output_data_source","input_vector","output_text" , "output_vector"]
  pair_table = pd.DataFrame([], columns=columns)
  for i in range(len(input_table)):
    vector = []
    for j in range(len(output_table)): 
      input_vector = input_table.iloc[i]['vector']
      output_vector = output_table.iloc[j]['vector']
      vector.append(abs(input_vector[0] @ output_vector[0].T))
    vector = np.array(vector)
    indexes = vector.argsort()[-k:]
    
    for index in indexes:
      output_data_source_index = str(output_data_source) + " => " + str(index + 1)
      pair_table = pair_table.append({'model':model_name,"output_data_source":output_data_source_index,"input_text":input_table.iloc[i]['input_text'],"input_vector":input_table.iloc[i]['vector'],"output_text":output_table.iloc[index]['output_text'] , "output_vector":output_table.iloc[index]['vector']}, ignore_index = True)
  
  return pair_table

     


In [ ]:
create_pair_knn(3,"LASER","تک ابیات",input_table, tak_beit_table)

Cosine KNN

In [ ]:
def create_pair_cosine_knn(k,model_name,output_data_source,input_table,output_table):
  columns = ["model","input_text","output_data_source","input_vector","output_text" , "output_vector"]
  pair_table = pd.DataFrame([], columns=columns)
  model = NearestNeighbors(n_neighbors=k,
                          metric='cosine',
                          algorithm='brute',
                          n_jobs=-1)
  x = []
  # Convert out put to array
  for i in range (0, len(tak_beit_table)):
    x.append(tak_beit_table["vector"][i][0])

  model.fit(x)

  for n in range(0, len(input_table)):
    v = input_table["vector"][n]
    for j in range(0, k):
      output_data_source_index = str(model.kneighbors(v, k, return_distance=False)[0][j])
      output_data_source_compelete = str(output_data_source) + " => " + output_data_source_index
      input_text = input_table['input_text'][n]
      output_text = tak_beit_table["output_text"][model.kneighbors(v, k, return_distance=False)[0][j]]
      output_vector = tak_beit_table["vector"][model.kneighbors(v, k, return_distance=False)[0][j]]
      pair_table = pair_table.append({'model':model_name,"output_data_source":output_data_source_compelete,"input_text":input_text,"input_vector":v ,"output_text":output_text , "output_vector":output_vector}, ignore_index = True)
  return pair_table


In [ ]:
create_pair_cosine_knn(3,"LASER","تک ابیات",input_table, tak_beit_table)